In [1]:
import pandas as pd

import sklearn #ML model building
from sklearn import svm
from sklearn import cluster
from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score

C:\Users\Bruno\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
catalog = pd.read_csv('inputs/catalog.csv', sep=',')
catalog.head(10)

,pid,current_price,original_price,category,sub_category,sub_sub_category
0,c9fe04e5097c087b6b6eeccc3adc4a142e14aa46,699.0,NaN,c1bd5fd999bd577743936898ada96496b547af3c,f08770a96fb546673053ab799f5ea9cada06c06a,2d2c44a2d8f18a6271f0e8057313af68a46d0f24
1,e00988f42f7fc5f14ec6a0e7905789110f23c5de,150.0,NaN,37b2661cc181c987f68770e43990152026920ba5,11e623a37e87cf7995c466723ec99688d55cae8c,11e623a37e87cf7995c466723ec99688d55cae8c
2,1e2e7c0d4082295728c1684a702cb43e1d332d3e,549.9,1049.9,c37df0ae71b97699a478def3001a3516a905a51d,f08770a96fb546673053ab799f5ea9cada06c06a,d6605426aaa703bf19874c0caac79a661b73de33
3,a279dd2284eb57533ca417c258ede0a0526a6f6e,0.0,0.0,c1bd5fd999bd577743936898ada96496b547af3c,6d9d48ae11ee1909235f31e9bfe5d36aa1462cb3,9a2e3cb56f6a1756fd35a4fd70172a67ecf13639
4,405b0362ae6ec149700164811b0e7773c8300e9d,79.9,0.0,c37df0ae71b97699a478def3001a3516a905a51d,78af0aac89e0f15a6c9fc70b5bff79d98c6dcc43,2541a58f702844477aab540e5df7b859a1e3d5de
5,d2df16bee9d37212d9dc0a9709c3a8579fedcfff,209.9,0.0,c37df0ae71b97699a478def3001a3516a905a51d,78af0aac89e0f15a6c9fc70b5bff79d98c6dcc43,d1e9767d8a0bd6d2bf67011d187d114cce67abb1
6,bdd7e95656924176860952506e72218a84e44e2b,0.0,0.0,c1bd5fd999bd577743936898ada96496b547af3c,78af0aac89e0f15a6c9fc70b5bff79d98c6dcc43,26efe347411a7070ef5cf75485203b58ee3cd3e7
7,daae65b7b1a2599bbe626aeb99c4a7ed4fab5a27,1389.9,0.0,c37df0ae71b97699a478def3001a3516a905a51d,8da1de08d7ee1f4f425c27a91b7fc3771dc66f74,54f708b3c394d2fb2ba687d4d1e5ed8804307f6f
8,11e952b51619add4cc796cd8d174b6e08eef1c2b,179.9,369.9,c37df0ae71b97699a478def3001a3516a905a51d,78af0aac89e0f15a6c9fc70b5bff79d98c6dcc43,eb0619cfb60da489f61001b19331dad5d480f82b
9,6af9bdadcdaef8b2654432c98b3c111d74c2dfc2,449.9,0.0,c37df0ae71b97699a478def3001a3516a905a51d,78af0aac89e0f15a6c9fc70b5bff79d98c6dcc43,d25e9813f29dcd86213b8879536008c79f1c80d2


In [3]:
#print(catalog.category.unique())
cat_keys = catalog.category.unique()
print(len(cat_keys))

#print(catalog.sub_category.unique())
sub_cat_keys = catalog.sub_category.unique()
print(len(sub_cat_keys))

#print(catalog.sub_sub_category.unique())
sub_sub_cat_keys = catalog.sub_sub_category.unique()
print(len(sub_sub_cat_keys))

#checking to see if there are any duplicates
print(len(catalog.pid.unique()))

print(len(catalog.pid))

14
33
81
19091
19091


In [4]:
#Initially will ignore sub_sub_category. Later will revisit to see if it improves accuracy
catalog = catalog.drop(['sub_sub_category'], axis=1)

#Making it easier to both study and parse with algos, by transforming the categories in simple numeric variables
cat_map = {}
sub_cat_map = {}

for i in range(0, len(cat_keys)):
    cat_map[cat_keys[i]] = i
    
for i in range(0, len(sub_cat_keys)):
    sub_cat_map[sub_cat_keys[i]] = i
    
catalog.category = catalog.category.map(cat_map)
catalog.sub_category = catalog.sub_category.map(sub_cat_map)

catalog.head(10)

,pid,current_price,original_price,category,sub_category
0,c9fe04e5097c087b6b6eeccc3adc4a142e14aa46,699.0,NaN,0,0
1,e00988f42f7fc5f14ec6a0e7905789110f23c5de,150.0,NaN,1,1
2,1e2e7c0d4082295728c1684a702cb43e1d332d3e,549.9,1049.9,2,0
3,a279dd2284eb57533ca417c258ede0a0526a6f6e,0.0,0.0,0,2
4,405b0362ae6ec149700164811b0e7773c8300e9d,79.9,0.0,2,3
5,d2df16bee9d37212d9dc0a9709c3a8579fedcfff,209.9,0.0,2,3
6,bdd7e95656924176860952506e72218a84e44e2b,0.0,0.0,0,3
7,daae65b7b1a2599bbe626aeb99c4a7ed4fab5a27,1389.9,0.0,2,4
8,11e952b51619add4cc796cd8d174b6e08eef1c2b,179.9,369.9,2,3
9,6af9bdadcdaef8b2654432c98b3c111d74c2dfc2,449.9,0.0,2,3


In [5]:
#Remove NaN values, so we can work with original_price and current_price
catalog.fillna(-1, axis=1, inplace=True)
catalog.head(10)

,pid,current_price,original_price,category,sub_category
0,c9fe04e5097c087b6b6eeccc3adc4a142e14aa46,699.0,-1.0,0,0
1,e00988f42f7fc5f14ec6a0e7905789110f23c5de,150.0,-1.0,1,1
2,1e2e7c0d4082295728c1684a702cb43e1d332d3e,549.9,1049.9,2,0
3,a279dd2284eb57533ca417c258ede0a0526a6f6e,0.0,0.0,0,2
4,405b0362ae6ec149700164811b0e7773c8300e9d,79.9,0.0,2,3
5,d2df16bee9d37212d9dc0a9709c3a8579fedcfff,209.9,0.0,2,3
6,bdd7e95656924176860952506e72218a84e44e2b,0.0,0.0,0,3
7,daae65b7b1a2599bbe626aeb99c4a7ed4fab5a27,1389.9,0.0,2,4
8,11e952b51619add4cc796cd8d174b6e08eef1c2b,179.9,369.9,2,3
9,6af9bdadcdaef8b2654432c98b3c111d74c2dfc2,449.9,0.0,2,3


In [6]:
#Create a column indicating if the product is on sale or not
catalog = catalog.assign(is_sale = lambda x: x.current_price < x.original_price)
catalog.head(10)

,pid,current_price,original_price,category,sub_category,is_sale
0,c9fe04e5097c087b6b6eeccc3adc4a142e14aa46,699.0,-1.0,0,0,False
1,e00988f42f7fc5f14ec6a0e7905789110f23c5de,150.0,-1.0,1,1,False
2,1e2e7c0d4082295728c1684a702cb43e1d332d3e,549.9,1049.9,2,0,True
3,a279dd2284eb57533ca417c258ede0a0526a6f6e,0.0,0.0,0,2,False
4,405b0362ae6ec149700164811b0e7773c8300e9d,79.9,0.0,2,3,False
5,d2df16bee9d37212d9dc0a9709c3a8579fedcfff,209.9,0.0,2,3,False
6,bdd7e95656924176860952506e72218a84e44e2b,0.0,0.0,0,3,False
7,daae65b7b1a2599bbe626aeb99c4a7ed4fab5a27,1389.9,0.0,2,4,False
8,11e952b51619add4cc796cd8d174b6e08eef1c2b,179.9,369.9,2,3,True
9,6af9bdadcdaef8b2654432c98b3c111d74c2dfc2,449.9,0.0,2,3,False


In [7]:
copy_catalog = catalog.copy(deep=True)
copy_catalog = copy_catalog.drop(['pid','current_price', 'original_price'], axis=1)

#copy_catalog.head(10)

cluster = cluster.KMeans(n_clusters=15)
cluster.fit(copy_catalog)
result = cluster.predict(copy_catalog)

In [8]:
print(result)
print(len(result))

[9 7 3 ..., 2 8 9]
19091


In [9]:
catalog = catalog.assign(cluster = result)
catalog.head(30)

,pid,current_price,original_price,category,sub_category,is_sale,cluster
0,c9fe04e5097c087b6b6eeccc3adc4a142e14aa46,699.0,-1.0,0,0,False,9
1,e00988f42f7fc5f14ec6a0e7905789110f23c5de,150.0,-1.0,1,1,False,7
2,1e2e7c0d4082295728c1684a702cb43e1d332d3e,549.9,1049.9,2,0,True,3
3,a279dd2284eb57533ca417c258ede0a0526a6f6e,0.0,0.0,0,2,False,5
4,405b0362ae6ec149700164811b0e7773c8300e9d,79.9,0.0,2,3,False,1
5,d2df16bee9d37212d9dc0a9709c3a8579fedcfff,209.9,0.0,2,3,False,1
6,bdd7e95656924176860952506e72218a84e44e2b,0.0,0.0,0,3,False,6
7,daae65b7b1a2599bbe626aeb99c4a7ed4fab5a27,1389.9,0.0,2,4,False,1
8,11e952b51619add4cc796cd8d174b6e08eef1c2b,179.9,369.9,2,3,True,10
9,6af9bdadcdaef8b2654432c98b3c111d74c2dfc2,449.9,0.0,2,3,False,1


In [10]:
catalog.to_csv('processed_catalog.csv', sep=';')